## Deze is niet afgemaakt, werkt niet

# Imports

In [23]:
import re
import pandas as pd
import numpy as np

# Data

In [24]:
bestand = r'.\data\\voorbeeld6.txt'

with open(bestand, 'r') as file:
    data = (file
            .read()
             .splitlines()
            )
    
df = (
    pd.DataFrame(data)[0]
    .str.split('', expand=True)
    .replace('', np.nan)
    .dropna(axis=1)
    .reset_index(drop=True)
)


# Dag 6.1

In [25]:
def find_initial_coordinate(df):
    indeces = np.nonzero(df.isin(['^', 'v', '>', '<']))
    row_index = indeces[0][0]
    col_index = indeces[1][0]
    inital_coordinates = (row_index, col_index)
    return inital_coordinates

In [26]:
def determine_look_direction(df, coordinates):
    row_index = coordinates[0]
    col_index = coordinates[1]
    if df.iloc[row_index, col_index] == '^':
        direction_looking = 'up'

    elif df.iloc[row_index, col_index] == '>':
        direction_looking = 'right'

    elif df.iloc[row_index, col_index] == 'v':
        direction_looking = 'down'

    elif df.iloc[row_index, col_index] == '<':
        direction_looking = 'left'

    else:
        raise Exception('coordinates are wrong, no looking position')
    
    new_df = df.copy()
    new_df.iloc[row_index, col_index] = '.'

    return new_df, direction_looking

In [27]:
def found_the_escape(df, coordinates, direction_looking):
    last_index_col = df.shape[1]-1
    last_index_row = df.shape[0]-1
    found_escape = False

    if (coordinates[0]==0) & (direction_looking == 'up'):
        found_escape = True
    
    if (coordinates[0]==last_index_row) & (direction_looking == 'down'):
        found_escape = True

    if (coordinates[1]==0) & (direction_looking == 'left'):
        found_escape = True
    
    if (coordinates[1]==last_index_col) & (direction_looking == 'right'):
        found_escape = True
    
    return found_escape
    

    



In [28]:
def define_direction(direction_looking):
    row_direction = 0
    col_direction = 0
    if direction_looking == 'up':
        row_direction = -1
    elif direction_looking == 'down':
        row_direction = 1
    elif direction_looking == 'right':
        col_direction = 1
    elif direction_looking == 'left':
        col_direction = -1

    return row_direction, col_direction

In [30]:
standing_ad_cooridinates = find_initial_coordinate(df)
direction_looking = 'up'
list_of_visits_including_facing = pd.DataFrame(columns=df.columns, index=df.index)
visits = pd.DataFrame(columns=df.columns, index=df.index)

# def walk_as_far_as_posible(df, standing_ad_cooridinates, direction_looking, list_of_visits_including_facing, visits):


row_index = standing_ad_cooridinates[0]
col_index = standing_ad_cooridinates[1]

found_escape = False
while found_escape == False:
    print(row_index, col_index)
    if direction_looking == 'up':
        serie_to_walk = df.iloc[:row_index, col_index]
        try:
            first_stop = max(np.nonzero(serie_to_walk.isin(["#"]))[0])
            visits.iloc[first_stop+1:row_index, col_index] = 'X'
            direction_looking = "right"
            row_index = first_stop + 1
        except: # no boundary
            visits.iloc[:row_index, col_index] = 'X'
            found_escape = True

    if direction_looking == "right":
        serie_to_walk = df.iloc[row_index, col_index:]
        try:
            first_stop = min(np.nonzero(serie_to_walk.isin(["#"]))[0]) + col_index
            visits.iloc[row_index, col_index:first_stop] = 'X'
            direction_looking = "down"
            col_index = first_stop - 1
        except:
            visits.iloc[row_index, col_index:] = 'X'
            found_escape = True
       
    if direction_looking == "down":
        serie_to_walk = df.iloc[row_index:, col_index]
        try:
            first_stop =  min(np.nonzero(serie_to_walk.isin(["#"]))[0]) + row_index
            visits.iloc[row_index:first_stop, col_index] = 'X'
            direction_looking = "left"
            row_index = first_stop - 1
        except:
            visits.iloc[row_index:, col_index] = 'X'
            found_escape = True
    
    if direction_looking == "left":
        serie_to_walk = df.iloc[row_index, :col_index]
        try:
            first_stop =  max(np.nonzero(serie_to_walk.isin(["#"]))[0]) 
            visits.iloc[row_index, first_stop:col_index] = 'X'
            direction_looking = "up"
            row_index = first_stop + 1
        except:
            visits.iloc[row_index, :col_index] = 'X'
            found_escape = True
    display(visits)
    print(direction_looking)
    print(row_index, col_index)


6 4


,1,2,3,4,5,6,7,8,9,10
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,X,X,X,X,X,NaN
2,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN,X,NaN
3,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN,X,NaN
4,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN,X,NaN
5,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN,X,NaN
6,NaN,X,X,X,X,X,X,X,X,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


up
2 8
2 8


,1,2,3,4,5,6,7,8,9,10
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,NaN
1,NaN,NaN,NaN,NaN,X,X,X,X,X,NaN
2,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN,X,NaN
3,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN,X,NaN
4,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN,X,NaN
5,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN,X,NaN
6,NaN,X,X,X,X,X,X,X,X,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


up
2 8


In [153]:
df
df.iloc[1,5:]

6     .
7     .
8     .
9     .
10    #
Name: 1, dtype: object

In [150]:
df

,1,2,3,4,5,6,7,8,9,10
0,.,.,.,.,#,.,.,.,.,.
1,.,.,.,.,.,.,.,.,.,#
2,.,.,.,.,.,.,.,.,.,.
3,.,.,#,.,.,.,.,.,.,.
4,.,.,.,.,.,.,.,#,.,.
5,.,.,.,.,.,.,.,.,.,.
6,.,#,.,.,^,.,.,.,.,.
7,.,.,.,.,.,.,.,.,#,.
8,#,.,.,.,.,.,.,.,.,.
9,.,.,.,.,.,.,#,.,.,.


In [128]:
df

,1,2,3,4,5,6,7,8,9,10
0,.,.,.,.,#,.,.,.,.,.
1,.,.,.,.,.,.,.,.,.,#
2,.,.,.,.,.,.,.,.,.,.
3,.,.,#,.,.,.,.,.,.,.
4,.,.,.,.,.,.,.,#,.,.
5,.,.,.,.,.,.,.,.,.,.
6,.,#,.,.,^,.,.,.,.,.
7,.,.,.,.,.,.,.,.,#,.
8,#,.,.,.,.,.,.,.,.,.
9,.,.,.,.,.,.,#,.,.,.


In [ ]:
def walk_as_far_as_posible(data, standing_ad_cooridinates, direction_looking, list_of_visits_including_facing, list_of_visits):
    row_direction, col_direction = define_direction(direction_looking)
    row = standing_ad_cooridinates[0]
    col = standing_ad_cooridinates[1]
    boundary_or_end_found = False
    in_loop = False

    i = 1
    j = 1
    while boundary_or_end_found == False:
        next_row = row+row_direction*i
        next_col = col+col_direction*j
        
        if (next_row < 0) | (next_col < 0):
            boundary_or_end_found == True
            break

        try:
            element_next = data[next_row][next_col]
        except:
            boundary_or_end_found == True
            break
        
        if element_next == '.':
            if (next_row, next_col) not in list_of_visits:
                list_of_visits.append((next_row, next_col))
            if (next_row, next_col, direction_looking) not in list_of_visits_including_facing:
                list_of_visits_including_facing.append((next_row, next_col, direction_looking))
            else:
                in_loop = True
                break
            i += 1
            j += 1
        elif element_next == '#':
            boundary_or_end_found = True

    
    new_coordinates = (row+row_direction*(i-1), col+col_direction*(j-1))  

    return list_of_visits_including_facing, list_of_visits, new_coordinates, in_loop

In [7]:
def turn(direction_looking):
    new_direction_looking = ''
    if direction_looking == 'up':
        new_direction_looking = 'right'
    elif direction_looking == 'right':
        new_direction_looking = 'down'
    elif direction_looking == 'down':
        new_direction_looking = 'left'
    elif direction_looking == 'left':
        new_direction_looking = 'up'
    
    return new_direction_looking

In [ ]:
def walk_and_count(df):
    standing_ad_cooridinates = find_initial_coordinate(df)
    df, direction_looking = determine_look_direction(df, standing_ad_cooridinates)
    found_escape = False
    visits = pd.DataFrame(columns=df.columns, index=df.index)
    visits_including_facing = pd.DataFrame(columns=df.columns, index=df.index)
    in_loop = False

    while (found_escape == False | in_loop == False):
        visits_including_facing, visits, standing_ad_cooridinates, in_loop = (
            walk_as_far_as_posible(
                df, 
                standing_ad_cooridinates, 
                direction_looking, 
                visits_including_facing,
                visits
                )
        )
        found_escape = found_the_escape(df, standing_ad_cooridinates, direction_looking)
        if found_escape == False:
            direction_looking = turn(direction_looking)
    
    number_of_visit_places = (visits == 'X').sum().sum()
    return number_of_visit_places, in_loop

In [ ]:
walk_and_count(df)

(5086, False)

In [15]:
d = pd.DataFrame(columns=df.columns, index=df.index)
d.iloc[0,0] = 'X'
d.iloc[3,3] = 'X'
(d == 'X').sum().sum()

2

# Dag 6.2

In [10]:
def make_new_data(data,row_number, col_number):
    new_data = data.copy()
    possible = True
    if len(re.findall("[<>v^#]", data[row_number][col_number]))>0:
           possible = False
    new_data[row_number] = new_data[row_number][:col_number]+'#'+data[row_number][col_number+1:]
    return new_data, possible

In [28]:
def count_possible_obstakels(data):
    total_inloop = 0
    for row_number in range(0,len(data)):
        for col_number in range(0,len(data[row_number])):
            new_data, possible = make_new_data(data, row_number, col_number)
            if possible:
                number_of_visits, in_loop = walk_and_count(new_data)
                if in_loop:
                    total_inloop += 1
        print(row_number)
    return total_inloop      



In [ ]:
count_possible_obstakels(data)

0
